# Domácí úkol


In [31]:
# Import necessary modules
import numpy as np
import matplotlib.pyplot as plt
import math
# Use LaTeX
plt.rcParams['text.usetex'] = True

## 1. Spinograf
- **Úkol : nakreslete obrázek trajektorie bodu na spinografu pro zadané hodnoty**
    - $R$ (poloměr větší kružnice),
    - $r$ (poloměr menší kružnice) a
    - $d$ (vzdálenost bodu od středu menší kružnice).
- Rozmyslete si, kolik rotací musí menší kružnice udělat, aby se bod vrátil do své původní pozice. (vykreslete dráhu tak, aby byla uzavřená)
- Vyzkoušejte alespoň pro $R=10, r=3, d=1$ a $R=7, r=2, d=5$.
- Vykreslete pomocí tenké modré čáry, zvolte dostatek bodů, aby byla trajektorie plynulá (hladká).

---

### Co je to spinograf?
Jedná se o hru/výukovou pomůcku (viz https://en.wikipedia.org/wiki/Spirograph).

Pro vysvětlení trajektorie bodu na spinografu (často označovaného jako hypotrochoida; když se menší kružnice otáčí uvnitř větší kružnice) použijeme goniometrický popis polohy bodu na kružnici v závislosti na úhlu rotace. Máme jednu větší kružnici s poloměrem $R$ a jednu menší kružnici s poloměrem $r$ a chceme trasovat dráhu bodu, který je od středu menší kružnice vzdálen $d$.


### Poloha středu menší kružnice

Střed menší kružnice se otáčí uvnitř větší kružnice. Poloha středu menší kružnice v závislosti na úhlu $\theta$ je dána parametrickými rovnicemi v kartézských souřadnicích:

-  $ X = (R - r)  \cos(\theta) $ 
-  $ Y = (R - r)  \sin(\theta) $ 

kde $\theta$ je úhel rotace středu menší kružnice v radiánech.

### Otočení menší kružnice

Menší kružnice se otočí (valí se po větší kružnici). Úhel, o který se menší kružnice otočila kolem svého středu vzhledem k počátečnímu stavu, je dán délkou oblouku, který urazila:
$$ \theta R $$

Pokud chceme tento oblouk vyjádřit v radiánech vzhledem k úhlu otočení, o který se otočila menší kružnice kolem svého středu, převádíme zpětně z oblouku na úhel:
$$ \theta \frac{R}{r} $$
Poslední dvě důležitá pozotování jsou:
- malá kružnice se otočí negativně (tedy úhel je záporný)
- malá kružnice se otočí zároveň se svým středem, tedy k úhlu otočení musíme přičíst úhel $\theta$

Celkově máme:

$$ \phi = \theta - \theta \frac{R}{r} = - \theta \frac{R-r}{r} $$


### Pozice bodu na spinografu

Nakonec sloučíme pozici středu a relativní pozici bodu ve vzdálenosti $d$ od středu menší kružnice:

-  $x = X + d \cos(\phi) = (R - r) \cos(\theta) + d \cos(- \theta \frac{R-r}{r})$ 
-  $y = Y + d \sin(\phi) = (R - r) \sin(\theta) + d \sin(- \theta \frac{R-r}{r})$ 

Pokud použijeme vlastnosti (sudost/lichost) gonimetricých funkcí, můžeme tyto rovnice upravit na:

$$ x = (R - r) \cos(\theta) + d \cos(\theta \frac{R-r}{r}) $$
$$ y = (R - r) \sin(\theta) - d \sin(\theta \frac{R-r}{r}) $$

In [60]:
# Draw a hypotrochoid
# param R is a radius of the greater circle
# param r is a radius of the smaller circle
# param d is a length of the arm on which the draw-point lays///////////////////////////////
def hypotrochoid(R=10, r=7, d=1):
    # ==== RANGE AND FINENESS OF PARAMETER Θ ====
    # Find the necessary angle θ_max to return to close the curve:
    # I base the algorithm for finding the θ_max on following assumptions:
    # - Let O be the perimeter of the greater circle G and o that of the smaller circle S and let ν be
    # the radius vector of the center of the circle S being positioned in the center of the circle G
    # and θ the angle of ν.
    # Then the hypotrochoid is closed (first time) if θ is such that $ n * O = m * o $
    # where n,m are natural numbers and $ θ = 2πn $ (n,m being the smallest possible).
    # The number n actually represents the number of revolutions of the radius vector ν (which is determined by θ).
    # - All numbers in a computer are rational, indeed - so irrational (for which a solution does not exist)
    # are excluded. Then the smallest solution of the equation $ n * O = m * o $ can be found this way:
    #   - Find a fractional form of O and o -> $ O = O_n/O_d $ and $ o = o_n/o_d $ respectively
    #   - Find the least common denominator (LCD) of those fractions (i.e. $lcd = lcm(O_d, o_d)$)
    #   - Find the numerators (O_N, o_N) respective to the LCD
    #   - Find their least common multiple (LCM)
    #   - Then $ n = LCM / O_n $
    # - To find the value of n with perimeters O and o, although more reasonable and clearer is this way, it is
    # computationally inefficient and unnecessary as O and o differ from known values of radii R and r respectively
    # by a factor 2π, and so the value of n can be calculated directly using the radii R and r instead of
    # the perimeters O and o.
    # - Knowing n, we can simply calculate θ_max from $ θ = 2πn $ (stated above)
    R_frac = R.as_integer_ratio()            # Tuple (numerator, denominator)
    r_frac = r.as_integer_ratio()            # Tuple (numerator, denominator)
    lcd = math.lcm(R_frac[1], r_frac[1])     # LCD of R and r
    R_numer = int(R * lcd)                   # Numerator for R with denominator lcd
    r_numer = int(r * lcd)                   # Numerator for r with denominator lcd
    numers_lcm = math.lcm(R_numer, r_numer)  # LCM of numerators respective to lcd
    revols_of_G = numers_lcm / R_numer       # Rovolutions of the greater circle G to close the curve
    θ_min = 0                                # Angle of the radius vector ν at the "starting point" of the curve
    θ_max = 2 * np.pi * revols_of_G          # The least angle θ necessary to close the curve
    θ_fineness = int(100 * revols_of_G)      # Fineness set to 100 points for each revolution

    # ==== DATA GENERATION ====
    θ = np.linspace(θ_min, θ_max, θ_fineness)  # Vector of values of θ
    λ = R - r  # Temporary value (distance between centers of circles)
    κ = λ / r  # Temporary value (koefficient of θ)
    φ = κ * θ  # Temporary variable for the second goniometrical function
    x = λ * np.cos(θ) + d * np.cos(φ)  # x-coordinate values for parameter θ
    y = λ * np.sin(θ) - d * np.sin(φ)  # y-coordinate values for parameter θ

    # ==== VISUALIZATION ====
    # Create a figure with axes
    fig = plt.figure(figsize=(8, 8), dpi=100)
    axes = fig.add_axes((0.1, 0.1, 0.8, 0.8))
    # Pass data to the figure
    axes.plot(x, y, color="blue", linewidth=1, linestyle='-')
    # Set texts
    axes.set_title(f"Spirograph (hypotrochoid) for ${R = }, {r = }, {d = }$",
                   fontsize=17,
                   family="serif")
    # Show the figure
    plt.show()

In [ ]:
# Tests
hypotrochoid(R=10, r=3, d=1)
hypotrochoid(R=7, r=2, d=5)
hypotrochoid()
hypotrochoid(R=20, r=7, d=2)
hypotrochoid(R=10, r=7, d=6)
hypotrochoid(R=10, r=7, d=8)
hypotrochoid(R=2.5, r=2, d=1)
# hypotrochoid(R=2.1, r=2, d=1)

# 2. Prvočíselná spirála


**Část 1.: Vykreslení "slunečnicových bodů"**

- Vykreslete tzv. "slunečnicové body". Tedy prvních `N` takových bodů.
- Vykreslete je tak, aby byl vzor dobře vidět (zvolte vhodné velikosti bodů)

##### Co jsou to slunečnicové body?
Poloměr $r$ a úhel $\theta$ pro každý bod (body číslovány přirozenými čísly: $n=1,...,N$) jsou určeny následovně:

1. Zlatý úhel $\phi$ je dán vztahem:
$$
\phi = \pi \cdot (3 - \sqrt{5})
$$

2. Poloměr $r$ je odvozen z indexu bodu $n$ takto:
$$
r = \sqrt{n}
$$

3. Úhel $\theta$ pro bod $n$ je vypočítán takto:
$$
\theta = n \cdot \phi
$$

Každý bod je poté umístěn do kartézských souřadnic $(x, y)$, které jsou vypočítány dle těchto vztahů:
$$
x = r \cdot \cos(\theta)
$$
$$
y = r \cdot \sin(\theta)
$$

**Část 2.: Vykreslení prvočíselné spirály**
- Vykreslete prvočíselnou spirálu pro prvních `N` prvočísel.
- Jedná se o stejný vzor jako v případě "slunečnicových bodů", ale tentokrát vykreslíme pouze ty body, které odpovídají prvočíslům.


In [ ]:
#TODO

# Bonusové úkoly
1. Modifikujte spinograf tak, aby:
    - menší kružnice se otáčela po vnější straně větší kružnice
    - menší kružnice se otáčela po rovnostraném n-úhelníku (trojúhelníku, čtverci, šestiúhelníku, ...)

In [ ]:
#TODO